In [1]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from pathlib import Path
import numpy as np, matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

# Configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", DEVICE)

# Your project structure
DATA_DIR = Path(r"D:\study\cos30082\final_asm\data\emotion")  # <-- your path
MODEL_DIR = Path(r"D:\study\cos30082\final_asm\models")
MODEL_DIR.mkdir(exist_ok=True)

IMG_SIZE = 48
BATCH_SIZE = 64

Running on: cpu


In [2]:
train_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

val_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_ds = datasets.ImageFolder(DATA_DIR / "train", transform=train_tf)
val_ds   = datasets.ImageFolder(DATA_DIR / "test",  transform=val_tf)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

num_classes = len(train_ds.classes)
print("Emotion classes:", train_ds.classes)


Emotion classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [3]:
from torchvision.models import resnet18, ResNet18_Weights

class EmotionNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        in_feats = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Linear(in_feats, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        return self.backbone(x)

emotion_model = EmotionNet(num_classes).to(DEVICE)
print("Emotion model ready.")


Emotion model ready.


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(emotion_model.parameters(), lr=1e-4)
EPOCHS = 10

best_val_acc = 0

for epoch in range(EPOCHS):
    emotion_model.train()
    running_loss, correct, total = 0, 0, 0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", ncols=100):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        logits = emotion_model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        preds = logits.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    train_acc = correct / total
    val_correct, val_total = 0, 0
    emotion_model.eval()
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            preds = emotion_model(imgs).argmax(1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total
    print(f"Epoch {epoch+1}: loss={running_loss/len(train_loader):.4f} | train_acc={train_acc:.4f} | val_acc={val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(emotion_model.state_dict(), MODEL_DIR / "emotion_detector_fer2013.pt")
        print("Saved best model with val_acc={:.4f}".format(best_val_acc))


Epoch 1/10: 100%|█████████████████████████████████████████████████| 449/449 [10:21<00:00,  1.38s/it]


Epoch 1: loss=1.4247 | train_acc=0.4511 | val_acc=0.5467
Saved best model with val_acc=0.5467


Epoch 2/10: 100%|█████████████████████████████████████████████████| 449/449 [03:11<00:00,  2.34it/s]


Epoch 2: loss=1.1251 | train_acc=0.5795 | val_acc=0.5894
Saved best model with val_acc=0.5894


Epoch 3/10: 100%|█████████████████████████████████████████████████| 449/449 [03:14<00:00,  2.31it/s]


...

Epoch 8: loss=0.4666 | train_acc=0.8373 | val_acc=0.6223
Saved best model with val_acc=0.6223


Epoch 9/10: 100%|█████████████████████████████████████████████████| 449/449 [03:56<00:00,  1.90it/s]


Epoch 9: loss=0.3905 | train_acc=0.8644 | val_acc=0.6197


Epoch 10/10: 100%|████████████████████████████████████████████████| 449/449 [03:56<00:00,  1.90it/s]


Epoch 10: loss=0.3227 | train_acc=0.8912 | val_acc=0.6194


In [6]:
import torch
model = torch.load(r"D:\study\cos30082\final_asm\models\emotion_detector_fer2013.pt", map_location="cpu")
print("Model loaded")


Model loaded
